In [1]:
from pandas import *  
import numpy as np
import re

import numpy as np
from scipy import stats
import scipy
from functools import reduce


In [2]:
df_emo_all = read_excel("./情感主观测试5-March14/情感主观测试5-March14-交付.xlsx")
df_emo_ref_all = read_excel("../25Feb/情感主观测试1-Feb25/情感主观测试1-Feb25-交付.xlsx")

df_vq_all = read_excel("./音质主观测试1-March14/音质主观测试1-March14-交付.xlsx")
df_vq_ref_all = read_excel("../25Feb/音质主观测试1-Feb25/音质主观测试1-Feb25-交付.xlsx")

In [3]:
emo_list = ['happy', 'sad', 'poor', 'fear', 'angry',  'amazed' , 'disgusted',   ]
spk_list = ['noise', 'TGLF']
exp_list = ['mix2_gvq_spkadv_105_pspk', 'mix2_gvq_spkadv_105_pspk_neu2',  'mix2_gvq_spkadv_105_pspk_neu']

In [4]:
def get_emotion(x):
    for emo in emo_list:
        if emo in x:
            return emo
        
def get_spk(x):
    for spk in spk_list:
        if spk in x:
            return spk

def get_exp(x):
    res_mix = re.search(r"mix2.*?%2Ft", x)
    if res_mix is None:
        return 'noise'
    return res_mix[0][:-4]

def get_fn(x):
    res_mix = re.search(r"TGEmotionalchild.*?.wav", x)
    if res_mix is None:
        return 'noise'
    return res_mix[0][:-4]

In [5]:
df_emo_all["score"] = df_emo_all["out"].map(lambda x:int(x[0]))
df_emo_all["emo"] = df_emo_all["link"].map(get_emotion)
df_emo_all["spk"] = df_emo_all["link"].map(get_spk)
df_emo_all['exp'] = df_emo_all['link'].map(get_exp)
df_emo_all['fn'] = df_emo_all['link'].map(get_fn)

df_emo_ref_all["score"] = df_emo_ref_all["out"].map(lambda x:int(x[0]))
df_emo_ref_all["emo"] = df_emo_ref_all["link"].map(get_emotion)
df_emo_ref_all["spk"] = df_emo_ref_all["link"].map(get_spk)
df_emo_ref_all['exp'] = df_emo_ref_all['link'].map(get_exp)
df_emo_ref_all['fn'] = df_emo_ref_all['link'].map(get_fn)

df_vq_all["score"] = df_vq_all["out"].map(lambda x:int(x[0]))
df_vq_all["emo"] = df_vq_all["link"].map(get_emotion)
df_vq_all["spk"] = df_vq_all["link"].map(get_spk)
df_vq_all['exp'] = df_vq_all['link'].map(get_exp)
df_vq_all['fn'] = df_vq_all['link'].map(get_fn)

df_vq_ref_all["score"] = df_vq_ref_all["out"].map(lambda x:int(x[0]))
df_vq_ref_all["emo"] = df_vq_ref_all["link"].map(get_emotion)
df_vq_ref_all["spk"] = df_vq_ref_all["link"].map(get_spk)
df_vq_ref_all['exp'] = df_vq_ref_all['link'].map(get_exp)
df_vq_ref_all['fn'] = df_vq_ref_all['link'].map(get_fn)

In [6]:
df_emo_ref = df_emo_ref_all[df_emo_ref_all['exp'] == 'mix2_gvq_spkadv_105_pspk']
df_emo_ref_new = df_emo_all[df_emo_all['exp'] == 'mix2_gvq_spkadv_105_pspk']

df_vq_ref = df_vq_ref_all[df_vq_ref_all['exp'] == 'mix2_gvq_spkadv_105_pspk']
df_vq_ref_new = df_vq_all[df_vq_all['exp'] == 'mix2_gvq_spkadv_105_pspk']



In [7]:
score_list = []
for num, exp_name in enumerate(exp_list):
    df_emo = df_emo_all[df_emo_all['exp'] == exp_name]
    df_vq = df_vq_all[df_vq_all['exp'] == exp_name]
    
    df_emo_score = df_emo.groupby('fn', as_index=False).mean().drop(columns=['id']).rename(columns={"score": "emo" + '_' + str(num)})
    df_vq_score = df_vq.groupby('fn', as_index=False).mean().drop(columns=['id']).rename(columns={"score": "vq" + '_' + str(num)})

    score_list.append(df_emo_score)
    score_list.append(df_vq_score)

    


In [8]:
df_merged = reduce(lambda  left,right: merge(left,right,on=['fn'],
                                            how='outer'), score_list)

In [9]:
df_merged['emo_1'] -= df_merged['emo_0']
df_merged['emo_2'] -= df_merged['emo_0']
df_merged['vq_1'] -= df_merged['vq_0']
df_merged['vq_2'] -= df_merged['vq_0']

In [10]:
df_merged = df_merged.drop(columns=['emo_0', 'vq_0'])

In [11]:
df_merged[(df_merged.vq_1 < 0) & (df_merged.vq_2 < 0)]

,fn,emo_1,vq_1,emo_2,vq_2
2,TGEmotionalchild_amazed_240175,-0.133333,-0.133333,-0.266667,-0.466667
3,TGEmotionalchild_amazed_240178,0.200000,-0.533333,0.066667,-0.066667
4,TGEmotionalchild_amazed_240215,0.200000,-0.400000,-0.133333,-0.800000
6,TGEmotionalchild_amazed_240339,-0.800000,-0.933333,-0.200000,-0.066667
13,TGEmotionalchild_angry_220129,0.333333,-0.066667,0.200000,-0.200000
20,TGEmotionalchild_disgusted_260257,-0.133333,-0.200000,0.066667,-0.066667
21,TGEmotionalchild_disgusted_260314,0.533333,-0.466667,0.200000,-0.133333
25,TGEmotionalchild_fear_210014,0.066667,-0.066667,-0.066667,-0.066667
28,TGEmotionalchild_fear_210065,0.000000,-0.133333,-0.066667,-0.200000
31,TGEmotionalchild_fear_210088,-0.266667,-0.066667,-0.200000,-0.066667


###  data 1

In [ ]:
for emo in emo_list:
    s_df = df_emo[df_emo['emo'] == emo]['score']
    s_df_ref = df_emo_ref[df_emo_ref['emo'] == emo]['score']
    s_df_ref_new = df_emo_ref_new[df_emo_ref_new['emo'] == emo]['score']
    
    m = np.mean(s_df) - np.mean(s_df_ref_new) + np.mean(s_df_ref)
    se = scipy.stats.sem(s_df)
    print(f"emo {emo} mean {round(m, 2) } \pm {round(scipy.stats.sem(s_df * scipy.stats.t.ppf((1 + 0.95) / 2., len(s_df)-1))/1.8, 2)}")


s_df = df_emo['score']
s_df_ref = df_emo_ref['score']
s_df_ref_new = df_emo_ref_new['score']
    
m = np.mean(s_df) - np.mean(s_df_ref_new) + np.mean(s_df_ref)
se = scipy.stats.sem(s_df)

print(f"mean {round(m, 2)} \pm {round(scipy.stats.sem(s_df * scipy.stats.t.ppf((1 + 0.95) / 2., len(s_df)-1))/1.8, 2)}")


In [ ]:
for emo in emo_list:
    new_df = df[df['emo'] == emo]
    new_df = new_df[new_df["score"] == 1]
    print(len(new_df.index.values))
    del_index = new_df.index.values
    df = df.drop(del_index)

###  sentence score

In [ ]:
df_emo_ref_all["link"][0]

In [ ]:
with pd.ExcelWriter('output.xlsx') as writer:  
    df1.to_excel(writer, sheet_name='Sheet_name_1')

In [ ]:
np.std(df['score'])

In [ ]:
df['score']